In [30]:
import numpy as np
import pandas as pd
from scipy import stats
from src.bayesian_network import BayesianNetwork

In [31]:
# Generate synthetic data
def generate_synthetic_data(n_samples=5000):
    A = np.random.normal(0, 1, n_samples)
    B = 0.5 * A + np.random.normal(0, 0.5, n_samples)
    C = 0.3 * A + 0.7 * B + np.random.normal(0, 0.3, n_samples)
    return pd.DataFrame({'A': A, 'B': B, 'C': C})

# Generate data
data = generate_synthetic_data()

# Create network
network = BayesianNetwork()

In [32]:
# Fit the network (this should create and connect nodes internally)
print("Fitting network...")
network.fit(data)

# Train network (if needed, depending on your implementation)
print("Training network...")
network.train(data, iterations=1000)

# Evaluate
print("\nEvaluating network...")
test_ll = network.log_likelihood(data)
cv_mean, cv_std = network.k_fold_cross_validation(data)  # Remove 'network' as the first argument

print(f"Test log-likelihood: {test_ll}")
print(f"Cross-validation: Mean LL = {cv_mean}, Std = {cv_std}")


Fitting network...
Training network...


Training Network:   0%|          | 1/1000 [00:00<03:00,  5.55it/s]

Training Network: 100%|██████████| 1000/1000 [00:41<00:00, 24.11it/s]



Evaluating network...
Test log-likelihood: -16738.315191952082
Cross-validation: Mean LL = -3349.738305699889, Std = 35.69537072026018


In [33]:
def test_sample_node():
    class SimpleNode:
        def __init__(self, name):
            self.name = name
            self.distribution = stats.norm(loc=0, scale=1)

        def sample(self, size=1):
            return self.distribution.rvs(size=size)

    class SimpleBayesianNetwork:
        def __init__(self):
            self.nodes = {
                'A': SimpleNode('A'),
                'B': SimpleNode('B')
            }

        def sample_node(self, node_name: str, size: int = 1) -> np.ndarray:
            print(f"sample_node called with node_name: {node_name}, size: {size}")
            node = self.nodes[node_name]
            return node.sample(size)

    bn = SimpleBayesianNetwork()
    try:
        print(bn.sample_node('A', size=5))  # Test sampling
    except Exception as e:
        print(f"Error: {e}")

test_sample_node()

sample_node called with node_name: A, size: 5
[ 0.97850129 -1.22266756 -1.45519578  0.5413156  -2.67503249]


In [36]:
# Sensitivity analysis
print("\nPerforming sensitivity analysis...")
sensitivity = network.sensitivity_analysis('C')
print("Sensitivity Analysis:", sensitivity)

# Sample from the network
print("\nSampling from the network...")
samples = {}
for node_name in network.nodes:
    print(f"Calling sample_node for {node_name} with 1000 samples")
    samples[node_name] = network.sample_node(node_name, size=1000)
for node_name, sample in samples.items():
    print(f"{node_name}: mean = {np.mean(sample):.4f}, std = {np.std(sample):.4f}")


ERROR:root:Error in sensitivity analysis for node A: BayesianNetwork.sample_node() takes from 1 to 2 positional arguments but 3 were given


ERROR:root:Error in sensitivity analysis for node B: BayesianNetwork.sample_node() takes from 1 to 2 positional arguments but 3 were given



Performing sensitivity analysis...
Sensitivity Analysis: {}

Sampling from the network...
Calling sample_node for A with 1000 samples


TypeError: BayesianNetwork.sample_node() got multiple values for argument 'size'

In [ ]:

# Compute feature importance
print("\nComputing feature importance...")
importance = network.feature_importance('C')
print("Feature Importance:", importance)

# Explain structure
print("\nExplaining network structure...")
structure = network.explain_structure()
print("Network Structure:", structure)

In [ ]:
# Explain prediction
print("\nExplaining prediction...")
observation = {'A': 0.5, 'B': 0.7}
explanation = network.explain_prediction('C', observation)
print(explanation)

# Compute marginal effect
print("\nComputing marginal effect...")
marginal_effect = network.marginal_effect('C', 'A')
print("Marginal effect of A on C:")
print(marginal_effect)

print("\nBayesian Network analysis complete!")